In [1]:
# install hmmer library
!sudo apt-get install hmmer
!sudo apt-get install hmmer-doc
!sudo apt-get install ncbi-blast+

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdivsufsort3
Suggested packages:
  hmmer-doc
The following NEW packages will be installed:
  hmmer libdivsufsort3
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,198 kB of archives.
After this operation, 7,621 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libdivsufsort3 amd64 2.0.1-5 [42.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 hmmer amd64 3.3.2+dfsg-1 [1,155 kB]
Fetched 1,198 kB in 1s (1,087 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend re

In [2]:
import pandas as pd
import random

## Get the 3D structure
CSV files of PF00014 domains are based on two different queries.

```
( Identifier = "PF00014" AND Annotation Type = "Pfam" ) AND Data Collection Resolution < 3 AND Polymer Entity Sequence Length = [ 50 - 80 ] AND Polymer Entity Mutation Count < 10
```
and
```
 ( Identifier = "PF00014" AND Annotation Type = "Pfam" ) AND Data Collection Resolution < 2 AND Polymer Entity Sequence Length = [ 50 - 80 ] AND Polymer Entity Mutation Count < 2
 ```
 the difference are the `Resolution(3Å vs 2Å)`, `Polymer Entity Mutation Count(10 vs 2)` and at the end `grouping the polymer entities with different sequence identity(100% vs 50%)`.

 The stricter criteria, with a resolution of 2Å, a mutation count of less than 2, and a sequence identity of 100%, may lead to a smaller sample size but potentially higher quality data, while the less strict criteria, with a resolution of 3Å, a mutation count of less than 10, and a sequence identity of 50%, could yield a larger sample size but with a risk of including lower quality data. With the stricter rules, we have obtained 14 samples, whereas with the less strict rule, we have collected 28 samples. Additionally, we aim to assess which set of criteria—either the more stringent or the less restrictive—ultimately yields superior results in terms of sample quality and relevance to our research objectives.

Now we can download the tabular CSV files which contain `Entity ID`, `Sequence`, `Auth Asym ID`

In [84]:
!wget -O strict_seq.csv "https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/strict.csv"
!wget -O not_strict_seq.csv "https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/not_strict.csv"

--2024-05-14 20:02:20--  https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/strict.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/heispv/bioinformatics/master/lab-of-bioinformatics/project_files/strict.csv [following]
--2024-05-14 20:02:20--  https://raw.githubusercontent.com/heispv/bioinformatics/master/lab-of-bioinformatics/project_files/strict.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1190 (1.2K) [text/plain]
Saving to: ‘strict_seq.csv’

strict_seq.csv      100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-05-14 20:02:20 (32.

## Clean csv file

In [85]:
import pandas as pd

def clean_csv_file(path: str, output_file_name: str, string_or_file: str = 'f', save_format: str = 'f') -> str or None:
    """
    Reads and cleans a CSV file, providing options to return the cleaned data as a string or save it into a file.

    Parameters:
        path (str): The path to the CSV file to be cleaned.
        output_file_name (str): The name of the output file. Defaults to "output_seq".
        string_or_file (str): Determines whether to return the cleaned data as a string ('s') or save it into a file ('f'). Defaults to 'f'.
        save_format (str): Determines the format for saving the data into a file. For keys ('k') or Fasta format ('f'). Defaults to 'f'.

    Returns:
        str or None: If the user chooses to get the results as a variable ('s'),
        the cleaned data is returned as a string. If the user chooses to save the
        results into a file ('f'), the cleaned data is saved into a file.
    """
    print('-' * 40)
    print(f'Reading CSV file from {path}...')
    df = pd.read_csv(path)
    print(f'Initial number of records: {len(df)}')
    print('-' * 40)

    # Drop rows with missing 'Entity ID' and update 'Entity ID' format
    df = df.dropna(subset=['Entity ID'])
    print(f'Number of records after dropping missing Entity ID: {len(df)}')
    df['Entity ID'] = df['Entity ID'].str.split('_').str[0] + ':' + df['Auth Asym ID']
    df = df.drop(columns=['Auth Asym ID'])
    df = df.reset_index(drop=True)

    # Option to return the cleaned data as a string
    if string_or_file == 's':
        cleaned_data = '\n'.join(df['Entity ID'].values)
        print('-' * 40)
        print('Returning cleaned data as a string.')
        return cleaned_data

    # Option to save the cleaned data to a file
    elif string_or_file == 'f':
        if save_format == 'f':
            output_path = output_file_name + '.fasta'
            with open(output_path, 'w') as file:
                for idx, row in df.iterrows():
                    file.write(f"> {row['Entity ID']}\n{row['Sequence']}\n")
            print(f'Data saved to {output_path}')
        elif save_format == 'k':
            output_path = output_file_name + '.txt'
            with open(output_path, 'w') as f:
                f.write('\n'.join(df['Entity ID'].values))
            print(f'Data saved to {output_path}')
        print('-' * 40)
        return None


In [86]:
clean_csv_file(
    path='strict_seq.csv',
    output_file_name='strict_seqs',
    string_or_file='f',
    save_format='f'
    )

----------------------------------------
Reading CSV file from strict_seq.csv...
Initial number of records: 30
----------------------------------------
Number of records after dropping missing Entity ID: 15
Data saved to strict_seqs.fasta
----------------------------------------


In [87]:
clean_csv_file(
    path='not_strict_seq.csv',
    output_file_name='not_strict_seqs',
    string_or_file='f',
    save_format='f'
    )

----------------------------------------
Reading CSV file from not_strict_seq.csv...
Initial number of records: 49
----------------------------------------
Number of records after dropping missing Entity ID: 28
Data saved to not_strict_seqs.fasta
----------------------------------------


In [88]:
!cat strict_seqs.fasta | head -n 10

> 1AAP:A
VREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSA
> 1KTH:A
ETDICKLPKDEGTCRDFILKWYYDPNTKSCARFWYGGCGGNENKFGSQKECEKVCAPV
> 1ZR0:B
PTGNNAEICLLPLDYGPCRALLLRYYYDRYTQSCRQFLYGGCEGNANNFYTWEACDDACWRIE
> 3BYB:A
KDRPDFCELPADTGPCRVRFPSFYYNPDEKKCLEFIYGGCEGNANNFITKEECESTCAA
> 3M7Q:B
EAEASICSEPKKVGRCKGYFPRFYFDSETGKCTPFIYGGCGGNGNNFETLHQCRAICRALG


We can use the same function which we used to clean the csv file, to only extract the ids of the sequences. these ids are going to be used as input files in the PDBeFold website to get Multiple Seqence Alignment based on Multiple Structure Alignemt.

In [89]:
# Get sequence ids as .txt file for the PDBeFold input
clean_csv_file(
    path='strict_seq.csv',
    output_file_name='strict_ids',
    string_or_file='f',
    save_format='k'
    )

----------------------------------------
Reading CSV file from strict_seq.csv...
Initial number of records: 30
----------------------------------------
Number of records after dropping missing Entity ID: 15
Data saved to strict_ids.txt
----------------------------------------


In [90]:
# Get sequence ids as .txt file for the PDBeFold input
clean_csv_file(
    path='not_strict_seq.csv',
    output_file_name='not_strict_ids',
    string_or_file='f',
    save_format='k'
    )

----------------------------------------
Reading CSV file from not_strict_seq.csv...
Initial number of records: 49
----------------------------------------
Number of records after dropping missing Entity ID: 28
Data saved to not_strict_ids.txt
----------------------------------------


Files `strict_ids.txt` and `not_strict_ids.txt` are uploaded into the [PDBeFold](https://www.ebi.ac.uk/msd-srv/ssm/cgi-bin/ssmserver) and the results are saved in my Github repository.

In [91]:
!rm not_strict_seq.csv strict_seq.csv

## Get MSA

In [92]:
# Getting the multiple sequence alignment from github repo
!wget -O strict_msa.txt "https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/strict_msa.txt"

--2024-05-14 20:03:09--  https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/strict_msa.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/heispv/bioinformatics/master/lab-of-bioinformatics/project_files/strict_msa.txt [following]
--2024-05-14 20:03:09--  https://raw.githubusercontent.com/heispv/bioinformatics/master/lab-of-bioinformatics/project_files/strict_msa.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1918 (1.9K) [text/plain]
Saving to: ‘strict_msa.txt’

strict_msa.txt      100%[===================>]   1.87K  --.-KB/s    in 0s      

2024-05-14 2

In [93]:
!wget -O not_strict_msa.txt "https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/not_strict_msa.txt"

--2024-05-14 20:03:11--  https://github.com/heispv/bioinformatics/raw/master/lab-of-bioinformatics/project_files/not_strict_msa.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/heispv/bioinformatics/master/lab-of-bioinformatics/project_files/not_strict_msa.txt [following]
--2024-05-14 20:03:11--  https://raw.githubusercontent.com/heispv/bioinformatics/master/lab-of-bioinformatics/project_files/not_strict_msa.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4029 (3.9K) [text/plain]
Saving to: ‘not_strict_msa.txt’

not_strict_msa.txt  100%[===================>]   3.93K  --.-KB/s    in 0s    

In [94]:
!rm strict_ids.txt not_strict_ids.txt

### Build HMM based on the raw MSA

In [95]:
# Create an HMM model based on the strict_msa.txt file
!hmmbuild strict_msa_not_clean.hmm strict_msa.txt

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             strict_msa.txt
# output HMM file:                  strict_msa_not_clean.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     strict_msa              15    63    57     2.35  0.976 

# CPU time: 0.05u 0.00s 00:00:00.05 Elapsed: 00:00:00.06


In [96]:
# Create an HMM model based on the not_strict_msa.txt file
!hmmbuild not_strict_msa_not_clean.hmm not_strict_msa.txt

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             not_strict_msa.txt
# output HMM file:                  not_strict_msa_not_clean.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     not_strict_msa          28    80    59     3.28  0.945 

# CPU time: 0.05u 0.00s 00:00:00.05 Elapsed: 00:00:00.06


In [97]:
!cat strict_msa_not_clean.hmm | head -n 22

HMMER3/f [3.3.2 | Nov 2020]
NAME  strict_msa
LENG  57
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Tue May 14 20:03:15 2024
NSEQ  15
EFFN  2.354736
CKSUM 1225978556
STATS LOCAL MSV       -8.8676  0.71902
STATS LOCAL VITERBI   -9.0620  0.71902
STATS LOCAL FORWARD   -4.0485  0.71902
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.60263  2.73612  3.07114  2.69296  2.83763  2.63193  3.88604  3.41319  2.66892  3.03622  4.07894  2.70680  3.44272  3.07560  2.87241  2.77771  2.91062  3.13755  4.62504  2.85507
          2.68622  4.42249  2.77475  2.73061  3.46378  2.40519  3.72518  3.29307  2.67748  2.69379  4.24714  2.90341  2.73739  3.18170  2.89777  2.37911  2.77518  2.98542  4.58501  3.61527
          0.72401  0.99536  1.92687  0.72737  0.66006  0.000

In [98]:
!cat not_strict_msa_not_clean.hmm | head -n 22

HMMER3/f [3.3.2 | Nov 2020]
NAME  not_strict_msa
LENG  59
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Tue May 14 20:03:15 2024
NSEQ  28
EFFN  3.284668
CKSUM 1929399890
STATS LOCAL MSV       -8.9836  0.71896
STATS LOCAL VITERBI   -9.1716  0.71896
STATS LOCAL FORWARD   -4.2058  0.71896
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.66045  2.92817  3.06444  2.66755  2.75405  2.64152  3.84570  3.40036  2.59333  3.03034  4.07046  2.68786  3.42610  3.06466  2.78686  2.80020  2.93294  3.20393  4.59062  2.86329
          2.68661  4.42268  2.77530  2.73007  3.46397  2.40541  3.72365  3.29271  2.67756  2.69368  4.24660  2.90390  2.73730  3.18114  2.89805  2.37919  2.77491  2.98562  4.58371  3.61546
          1.18984  1.11301  1.00195  1.14078  0.38503  0

Based on the files above, we can observe that the `hmmbuild` command, applied to the `strict_msa.txt` file, cuts the first `5` characters in the sequence while when using the `not_strict_msa.txt` it cuts the first `20` character. This action is taken because there are not enough amino acids to build the Hidden Markov Model (HMM) for that part of the sequence. Therefore, we will trim each sequence and then reapply the `hmmbuild` command.

## Clean raw MSA

In [99]:
def clean_msa(path: str, first_clipping_num: int, output_file_name: str) -> None:
    """
    Clean MSA file by removing specified number of characters from the beginning of each sequence.

    Args:
        path (str): Path to the input MSA file.
        first_clipping_num (int): Number of characters to remove from the beginning of each sequence.
        output_file_name (str): Name of the output file.

    Returns:
        None

    This function reads a MSA file, extracts the sequence IDs and sequences, removes the specified
    number of characters from the beginning of each sequence, and writes the cleaned sequences to a new file.
    """
    print('-' * 40)
    print(f'Reading MSA file from {path}...')
    with open(path) as f:
        fastas = f.read().strip().split('\n\n')
    print(f'Number of sequences found: {len(fastas)}')
    print('-' * 40)

    clean_list = []
    for fasta in fastas:
        id = fasta.split()[0]
        sequence = ''.join(fasta.split('\n')[1:])
        clean_list.append((id, sequence))

    print(f'Removing the first {first_clipping_num} characters from each sequence...')
    with open(output_file_name + '.txt', 'w') as f:
        for item in clean_list:
            f.write(f"{item[0]}\n{item[1][first_clipping_num:]}\n")
    print('-' * 40)

    print(f'Output saved in {output_file_name}.txt')
    print('-' * 40)


In [100]:
clean_msa(
    path='strict_msa.txt',
    first_clipping_num=5,
    output_file_name='clean_strict_msa'
    )

----------------------------------------
Reading MSA file from strict_msa.txt...
Number of sequences found: 15
----------------------------------------
Removing the first 5 characters from each sequence...
----------------------------------------
Output saved in clean_strict_msa.txt
----------------------------------------


In [101]:
clean_msa(
    path='not_strict_msa.txt',
    first_clipping_num=20,
    output_file_name='clean_not_strict_msa'
    )

----------------------------------------
Reading MSA file from not_strict_msa.txt...
Number of sequences found: 28
----------------------------------------
Removing the first 20 characters from each sequence...
----------------------------------------
Output saved in clean_not_strict_msa.txt
----------------------------------------


In [102]:
# Check the strict_msa.tx file
!cat clean_strict_msa.txt | head -n 20

>PDB:1aap:A
revcseqaetgpcrAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCg---
>PDB:1kth:A
tdicklpkdegtcrDFILKWYYDPNTKSCARFWYGGCGGNENKFGSQKECEKVCapv-
>PDB:1zr0:B
aeicllpldygpcrALLLRYYYDRYTQSCRQFLYGGCEGNANNFYTWEACDDACwrie
>PDB:3byb:A
pdfcelpadtgpcrVRFPSFYYNPDEKKCLEFIYGGCEGNANNFITKEECESTCa---
>PDB:3m7q:B
asicsepkkvgrckGYFPRFYFDSETGKCTPFIYGGCGGNGNNFETLHQCRAICralg
>PDB:3wny:C
pafcleppyagpgkARIIRYFYNAKAGAAQAFVYGGVRAKRNNFASAADALAACaa--
>PDB:4dtg:K
pdfcfleedpgicrGYITRYFYNNQTKQCERFKYGGCLGNMNNFETLEECKNICedgh
>PDB:4ntw:B
afcyedppffqkcgAFVDSYYFNRSRITCVHFFYGQCDVNQNHFTTMSECNRVChg--
>PDB:4u30:X
-acanlpivrgpcrAFIQLWAFDAVKGKCVLFPYGGCQGNGNKFYSEKECREYCg---
>PDB:4u32:X
hdfclvskvvgrcrASMPRWWYNVTDGSCQLFVYGGCDGNSNNYLTKEECLKKC----


In [103]:
# Check the not_strict_msa.tx file
!cat clean_not_strict_msa.txt | head -n 20

>PDB:1aap:A
vrevcseqaetgpcrAMISRWYFDVTEGKCAPFFYGGcGG-NRNNFDTEEYCMAVCg---
>PDB:1bun:B
rhpdcdkppdtkicqTVVRAFYYKPSAKRCVQFRYGG-CNgNGNHFKSDHLCRCECleyr
>PDB:1dtx:A
rrklcilhrnpgrcyDKIPAFYYNQKKKQCERFDWSGcGG-NSNRFKTIEECRRTCig--
>PDB:1fak:I
apdfcleppydgpcrALHLRYFYNAKAGLCQTFYYGGcLA-KRNNFESAEDCMRTC----
>PDB:1g6x:A
rpdfcleppyagacrARIIRYFYNAKAGLCQTFVYGGcRA-KRNNFKSAEDCLRTCgga-
>PDB:1kth:A
etdicklpkdegtcrDFILKWYYDPNTKSCARFWYGGcGG-NENKFGSQKECEKVCapv-
>PDB:1tfx:C
kpdfcfleedpgicrGYITRYFYNNQTKQCERFKYGGcLG-NMNNFETLEECKNICedg-
>PDB:1yc0:I
tedyclasnkvgrcrGSFPRWYYDPTEQICKSFVYGGcLG-NKNNYLREEECILACrgv-
>PDB:1ylc:B
rpdfxleppytgpckARIIRYFYNAKAGLXQTFVYGGcRA-KRNNFKSAEDXMRTXg---
>PDB:1yld:B
rpdfxleppytgpckARIIRYFYNAPDGLXQTFVYGGcRA-KRNNFKSAEDXMRTXg---


In [104]:
!rm not_strict_msa.txt not_strict_msa_not_clean.hmm strict_msa.txt strict_msa_not_clean.hmm

### Build HMM based on clean MSA

In [105]:
# Create an HMM model based on the clean_strict_msa.txt file
!hmmbuild strict_msa.hmm clean_strict_msa.txt

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             clean_strict_msa.txt
# output HMM file:                  strict_msa.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     clean_strict_msa        15    58    56     2.38  0.994 

# CPU time: 0.03u 0.00s 00:00:00.03 Elapsed: 00:00:00.03


In [106]:
# Create an HMM model based on the clean_not_strict_msa.txt file
!hmmbuild not_strict_msa.hmm clean_not_strict_msa.txt

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             clean_not_strict_msa.txt
# output HMM file:                  not_strict_msa.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     clean_not_strict_msa    28    60    58     3.31  0.961 

# CPU time: 0.04u 0.00s 00:00:00.04 Elapsed: 00:00:00.04


In [107]:
!cat strict_msa.hmm | head -n 22

HMMER3/f [3.3.2 | Nov 2020]
NAME  clean_strict_msa
LENG  56
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Tue May 14 20:03:36 2024
NSEQ  15
EFFN  2.384033
CKSUM 2703071005
STATS LOCAL MSV       -8.8221  0.71901
STATS LOCAL VITERBI   -9.0502  0.71901
STATS LOCAL FORWARD   -4.0496  0.71901
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.59850  2.70859  3.07791  2.70650  2.81735  2.62294  3.89445  3.41061  2.67810  3.03829  4.07877  2.70340  3.46464  3.08028  2.90361  2.78551  2.91215  3.14231  4.61766  2.83458
          2.68618  4.42225  2.77519  2.73123  3.46354  2.40513  3.72494  3.29354  2.67741  2.69355  4.24690  2.90347  2.73739  3.18146  2.89801  2.37887  2.77519  2.98518  4.58477  3.61503
          0.10417  4.75186  2.40473  0.61958  0.77255 

In [108]:
!cat not_strict_msa.hmm | head -n 22

HMMER3/f [3.3.2 | Nov 2020]
NAME  clean_not_strict_msa
LENG  58
ALPH  amino
RF    no
MM    no
CONS  yes
CS    no
MAP   yes
DATE  Tue May 14 20:03:37 2024
NSEQ  28
EFFN  3.312012
CKSUM 1133647359
STATS LOCAL MSV       -8.9546  0.71898
STATS LOCAL VITERBI   -9.1394  0.71898
STATS LOCAL FORWARD   -4.2119  0.71898
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.66190  2.90256  3.08226  2.67195  2.73539  2.63859  3.85121  3.39815  2.59820  3.03714  4.06723  2.68399  3.44414  3.07780  2.78809  2.81204  2.93638  3.20605  4.58366  2.84542
          2.68618  4.42225  2.77519  2.73123  3.46354  2.40513  3.72494  3.29354  2.67741  2.69355  4.24690  2.90347  2.73739  3.18146  2.89801  2.37887  2.77519  2.98518  4.58477  3.61503
          0.12720  5.00516  2.18269  0.61958  0.77

* In these new files, we can observe that the probabilities start from the first amino acid (AA), indicating that no cutting is performed by the `hmmbuild` command itself.

In [28]:
# Works only on my own Drive
#!cp /content/drive/MyDrive/lab_of_bioinformatics/negative.fasta /content/negative.fasta

## Get the negative and postive data from NCBI

In [ ]:
!wget -O negative.fasta.gz "https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28reviewed%3Atrue%29+NOT+%28xref%3Apfam-PF00014%29%29"
!zcat -f negative.fasta.gz > negative.fasta
!rm negative.fasta.gz

--2024-05-13 19:39:45--  https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28reviewed%3Atrue%29+NOT+%28xref%3Apfam-PF00014%29%29
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘negative.fasta.gz’

negative.fasta.gz       [        <=>         ] 134.34M   159KB/s    in 14m 47s 

2024-05-13 19:54:33 (155 KB/s) - ‘negative.fasta.gz’ saved [140864598]



In [109]:
# It contains the human and non-human kunitz domain
!wget -O bpti_reviewd.fasta.gz "https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28xref%3Apfam-PF00014%29+AND+%28reviewed%3Atrue%29%29"
!zcat bpti_reviewd.fasta.gz > bpti_reviewd.fasta
!rm bpti_reviewd.fasta.gz

--2024-05-14 20:03:44--  https://rest.uniprot.org/uniprotkb/stream?compressed=true&format=fasta&query=%28%28xref%3Apfam-PF00014%29+AND+%28reviewed%3Atrue%29%29
Resolving rest.uniprot.org (rest.uniprot.org)... 193.62.193.81
Connecting to rest.uniprot.org (rest.uniprot.org)|193.62.193.81|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘bpti_reviewd.fasta.gz’

bpti_reviewd.fasta.     [    <=>             ]  41.28K  58.7KB/s    in 0.7s    

2024-05-14 20:03:45 (58.7 KB/s) - ‘bpti_reviewd.fasta.gz’ saved [42270]



In [110]:
# Make blast dataset for the strict_seqs.fasta and not_strict_seqs.fasta
!makeblastdb -in strict_seqs.fasta -dbtype prot
!makeblastdb -in not_strict_seqs.fasta -dbtype prot



Building a new DB, current time: 05/14/2024 20:03:45
New DB name:   /content/strict_seqs.fasta
New DB title:  strict_seqs.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /content/strict_seqs.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 15 sequences in 0.00216484 seconds.




Building a new DB, current time: 05/14/2024 20:03:45
New DB name:   /content/not_strict_seqs.fasta
New DB title:  not_strict_seqs.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /content/not_strict_seqs.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 28 sequences in 0.00173903 seconds.




In [111]:
!cat bpti_reviewd.fasta | grep ">" | wc

    391    3918   40444


The line of the code below initiates a BLASTP search, a tool for comparing protein sequences. It takes protein sequences from "output_seq.fasta" as the query and compares them against a database specified in "bpti_reviewd.fasta". The results are saved in "bpti_reviewd.blast" using format 7, which is suitable for further analysis.

In [112]:
!blastp -query bpti_reviewd.fasta -db strict_seqs.fasta -out strict_bpti.blast -outfmt 7
!blastp -query bpti_reviewd.fasta -db not_strict_seqs.fasta -out not_strict_bpti.blast -outfmt 7

In [113]:
!cat strict_bpti.blast | head -n 40

# BLASTP 2.12.0+
# Query: sp|A0A1Z0YU59|MAMB1_DENAN Mambaquaretin-1 OS=Dendroaspis angusticeps OX=8618 PE=1 SV=2
# Database: strict_seqs.fasta
# Fields: query acc.ver, subject acc.ver, % identity, alignment length, mismatches, gap opens, q. start, q. end, s. start, s. end, evalue, bit score
# 15 hits found
sp|A0A1Z0YU59|MAMB1_DENAN	5M4V:A	94.737	57	3	0	1	57	1	57	1.42e-39	113
sp|A0A1Z0YU59|MAMB1_DENAN	3BYB:A	52.632	57	27	0	1	57	3	59	5.11e-21	66.2
sp|A0A1Z0YU59|MAMB1_DENAN	4DTG:K	43.636	55	31	0	1	55	3	57	6.55e-19	61.2
sp|A0A1Z0YU59|MAMB1_DENAN	1ZR0:B	45.283	53	29	0	3	55	7	59	3.89e-17	56.6
sp|A0A1Z0YU59|MAMB1_DENAN	5PTI:A	42.105	57	33	0	1	57	1	57	7.87e-16	53.1
sp|A0A1Z0YU59|MAMB1_DENAN	1KTH:A	38.889	54	33	0	3	56	3	56	1.02e-14	50.4
sp|A0A1Z0YU59|MAMB1_DENAN	4U30:X	42.000	50	29	0	6	55	7	56	1.66e-14	49.7
sp|A0A1Z0YU59|MAMB1_DENAN	6YHY:A	43.636	55	31	0	1	55	2	56	5.91e-14	48.5
sp|A0A1Z0YU59|MAMB1_DENAN	1AAP:A	39.623	53	32	0	3	55	3	55	8.01e-13	45.4
sp|A0A1Z0YU59|MAMB1_DENAN	6Q61:A	37.500	56	35	

This command below filters a BLAST result file named "bpti_reviewd.blast". It removes comment lines (lines starting with # character), selects entries with sequence identity greater than 98%, and saves the unique identifiers of those entries into a file named "remove.fasta".

In [114]:
!grep -v "^#" strict_bpti.blast | awk '{if ($3 > 98) {print $0}}' | cut -f 1 | sort -u > strict_remove_98.ids
!grep -v "^#" not_strict_bpti.blast | awk '{if ($3 > 98) {print $0}}' | cut -f 1 | sort -u > not_strict_remove_98.ids

In [115]:
!grep -v "^#" strict_bpti.blast | awk '{if ($3 > 80) {print $0}}' | cut -f 1 | sort -u > strict_remove_80.ids
!grep -v "^#" not_strict_bpti.blast | awk '{if ($3 > 80) {print $0}}' | cut -f 1 | sort -u > not_strict_remove_80.ids

In [116]:
!cat strict_remove_98.ids | wc
!cat not_strict_remove_98.ids | wc

     21      21     424
     28      28     577


In [117]:
!cat strict_remove_80.ids | wc
!cat not_strict_remove_80.ids | wc

     63      63    1346
     72      72    1542


In [118]:
!cat strict_remove_80.ids | head -n 5

sp|A0A1Z0YU59|MAMB1_DENAN
sp|A0A3G2FQK2|3DPD_HETMG
sp|A0A6B7FA07|VKT2J_HETCR
sp|A0A6B7FBD3|VKT2I_HETCR
sp|A0A6B7FEJ3|VKT2K_HETCR


We are only interested in the ids, to get the ids from the remove.fasta file, we should run the command below. the results would be saved in the `remove.ids` file.

Based on the `remove.ids` file, there are 27 sequences which should be removed from the main data.

In [120]:
def filter_sequences(seq_file_path, ids_file_path, output_file_path):
    """
    Filters sequences from a FASTA file based on a list of excluded sequence IDs and saves them in a file.

    Parameters:
    - seq_file_path (str): The file path to the input FASTA file containing sequences to filter.
    - ids_file_path (str): The file path to the input file containing a list of sequence IDs to exclude.
    - output_file_path (str): The file path to save the filtered sequences.

    Returns:
    - None
    """
    print('-' * 40)
    print(f'Reading excluded sequence IDs from {ids_file_path}...')
    # Open the file containing excluded sequence IDs and create a set to store them
    with open(ids_file_path, 'r') as f:
        excluded_ids = [line.strip().split('|')[1] for line in f]
    print(f'Number of excluded IDs: {len(excluded_ids)}')
    print('-' * 40)

    print(f'Reading sequences from {seq_file_path}...')
    # Open the input FASTA file and extract sequences
    with open(seq_file_path, 'r') as f:
        content = f.read().strip()
        sequences = content.split('>')[1:]
    print(f'Number of sequences found: {len(sequences)}')
    print('-' * 40)

    print(f'Filtering sequences and writing to {output_file_path}...')
    # Open the output file for writing filtered sequences
    with open(output_file_path, 'w') as outfile:
        filtered_count = 0
        for sequence in sequences:
            header = sequence.split('\n', 1)[0]
            seq_id = header.split('|')[1]

            if seq_id not in excluded_ids:
                outfile.write(f'>{sequence}\n')
                filtered_count += 1
    print(f'Number of sequences written to output: {filtered_count}')
    print('-' * 40)
    print('Filtering process completed.')


In [122]:
filter_sequences('bpti_reviewd.fasta', 'strict_remove_98.ids', 'strict_pos_98.fasta')

----------------------------------------
Reading excluded sequence IDs from strict_remove_98.ids...
Number of excluded IDs: 21
----------------------------------------
Reading sequences from bpti_reviewd.fasta...
Number of sequences found: 391
----------------------------------------
Filtering sequences and writing to strict_pos_98.fasta...
Number of sequences written to output: 370
----------------------------------------
Filtering process completed.


In [123]:
filter_sequences('bpti_reviewd.fasta', 'strict_remove_80.ids', 'strict_pos_80.fasta')

----------------------------------------
Reading excluded sequence IDs from strict_remove_80.ids...
Number of excluded IDs: 63
----------------------------------------
Reading sequences from bpti_reviewd.fasta...
Number of sequences found: 391
----------------------------------------
Filtering sequences and writing to strict_pos_80.fasta...
Number of sequences written to output: 328
----------------------------------------
Filtering process completed.


In [124]:
filter_sequences('bpti_reviewd.fasta', 'not_strict_remove_98.ids', 'not_strict_pos_98.fasta')

----------------------------------------
Reading excluded sequence IDs from not_strict_remove_98.ids...
Number of excluded IDs: 28
----------------------------------------
Reading sequences from bpti_reviewd.fasta...
Number of sequences found: 391
----------------------------------------
Filtering sequences and writing to not_strict_pos_98.fasta...
Number of sequences written to output: 363
----------------------------------------
Filtering process completed.


In [125]:
filter_sequences('bpti_reviewd.fasta', 'not_strict_remove_80.ids', 'not_strict_pos_80.fasta')

----------------------------------------
Reading excluded sequence IDs from not_strict_remove_80.ids...
Number of excluded IDs: 72
----------------------------------------
Reading sequences from bpti_reviewd.fasta...
Number of sequences found: 391
----------------------------------------
Filtering sequences and writing to not_strict_pos_80.fasta...
Number of sequences written to output: 319
----------------------------------------
Filtering process completed.


In [127]:
!cat strict_pos_98.fasta | head -n 5

>sp|A0A1Z0YU59|MAMB1_DENAN Mambaquaretin-1 OS=Dendroaspis angusticeps OX=8618 PE=1 SV=2
RPSFCNLPVKPGPCNGFFSAFYYSQKTNKCHSFTYGGCKGNANRFSTIEKCRRTCVG

>sp|B2G331|VKT2B_HETCR TauPI-stichotoxin-Hcr2b OS=Heteractis crispa OX=175771 PE=1 SV=1
MKGTFLICLILIAGFSFKSTQAGSICLEPKVVGPCTAYFRRFYFDSETGKCTVFIYGGCE


We want to have a simplified version of these fasta files...

In [131]:
def simplify_fasta(file_path):
    # Read the input file
    with open(file_path, 'r') as f:
        data = f.read()

    simplified = []
    entries = [entry for entry in data.split('>sp') if entry.strip()]

    for entry in entries:
        lines = entry.split('\n')
        # Check if '|' is in the first line and if not, handle gracefully
        identifier = lines[0].split('|')[1] if '|' in lines[0] else lines[0]

        # Join all lines except the first to get the sequence, and remove new lines
        sequence = ''.join(lines[1:]).replace('\n', '')
        simplified_entry = f'>{identifier}\n{sequence}'
        simplified.append(simplified_entry)

    # Create the output content
    output_content = '\n\n'.join(simplified)

    # Create the output file name
    output_file_path = file_path.rsplit('.', 1)[0] + '_simple.' + file_path.rsplit('.', 1)[1]

    # Write the simplified content to the new file
    with open(output_file_path, 'w') as output_file:
        output_file.write(output_content)

    print(f'File saved as {output_file_path}')
    return output_file_path  # Return the output file path

In [132]:
simplify_fasta("strict_pos_80.fasta")

File saved as strict_pos_80_simple.fasta


'strict_pos_80_simple.fasta'

In [133]:
def shuffle_fasta(data):
    import random
    # Split the data into entries using a marker that identifies new entries starting with '>'
    # This time, we split directly on the '>' and preserve it in the next step
    entries = data.strip().split('\n>')

    # Attach '>' back to each entry except the first one if it was removed
    entries = [entry if entry.startswith('>') else '>' + entry for entry in entries]

    # Shuffle the entries list randomly
    random.shuffle(entries)

    # Join all shuffled entries with a single newline between them, after ensuring each ends cleanly
    return '\n\n'.join(entry.strip() for entry in entries)

In [140]:
def process_fasta_files(positive_path, negative_path, training_portion):
    # Simplify the data
    print('-' * 40)
    print(f'Simplifying the data for {positive_path} ...')
    positive_simplified_path = simplify_fasta(positive_path)
    print(f'Simplifying the data for {negative_path} ...')
    negative_simplified_path = simplify_fasta(negative_path)
    print('-' * 40)

    # Read the simplified content
    print(f'Reading simplified data from {positive_simplified_path} ...')
    with open(positive_simplified_path, 'r') as file:
        simplified_positive = file.read()
    print(f'Reading simplified data from {negative_simplified_path} ...')
    with open(negative_simplified_path, 'r') as file:
        simplified_negative = file.read()
    print('-' * 40)

    # Shuffle the data
    print('Shuffling the positive sequences ...')
    shuffled_positive = shuffle_fasta(simplified_positive)
    print('Shuffling the negative sequences ...')
    shuffled_negative = shuffle_fasta(simplified_negative)
    print('-' * 40)

    # Split the data into entries
    pos_entries = shuffled_positive.split('\n\n')
    neg_entries = shuffled_negative.split('\n\n')

    print(f'Number of positive sequences: {len(pos_entries)}')
    print(f'Number of negative sequences: {len(neg_entries)}')
    print('-' * 40)

    # Determine the index for splitting based on the training_portion
    split_index_pos = int(len(pos_entries) * training_portion)
    split_index_neg = int(len(neg_entries) * training_portion)

    print(f'Training portion: {training_portion * 100}%')
    print(f'Number of positive sequences in training set: {split_index_pos}')
    print(f'Number of positive sequences in test set: {len(pos_entries) - split_index_pos}')
    print(f'Number of negative sequences in training set: {split_index_neg}')
    print(f'Number of negative sequences in test set: {len(neg_entries) - split_index_neg}')
    print('-' * 40)

    # Create training and testing datasets
    train_data = pos_entries[:split_index_pos] + neg_entries[:split_index_neg]
    test_data = pos_entries[split_index_pos:] + neg_entries[split_index_neg:]

    # Shuffle the final datasets
    random.shuffle(train_data)
    random.shuffle(test_data)

    # Extract the base name without extension from the positive path and remove '_pos' if it exists
    base_name = positive_path.rsplit('.', 1)[0]
    base_name = base_name.replace('_pos', '')

    # Save the datasets to files
    train_file_name = f'train_{base_name}.fasta'
    test_file_name = f'test_{base_name}.fasta'

    print(f'Saving training data to {train_file_name}')
    with open(train_file_name, 'w') as file:
        file.write('\n\n'.join(train_data))
    print(f'Saving test data to {test_file_name}')
    with open(test_file_name, 'w') as file:
        file.write('\n\n'.join(test_data))

    print('Training and testing datasets have been saved.')
    print('-' * 40)


In [141]:
process_fasta_files(
    positive_path="not_strict_pos_98.fasta",
    negative_path="negative.fasta",
    training_portion=0.8
    )

----------------------------------------
Simplifying the data for not_strict_pos_98.fasta ...
File saved as not_strict_pos_98_simple.fasta
Simplifying the data for negative.fasta ...
File saved as negative_simple.fasta
----------------------------------------
Reading simplified data from not_strict_pos_98_simple.fasta ...
Reading simplified data from negative_simple.fasta ...
----------------------------------------
Shuffling the positive sequences ...
Shuffling the negative sequences ...
----------------------------------------
Number of positive sequences: 363
Number of negative sequences: 570891
----------------------------------------
Training portion: 80.0%
Number of positive sequences in training set: 290
Number of positive sequences in test set: 73
Number of negative sequences in training set: 456712
Number of negative sequences in test set: 114179
----------------------------------------
Saving training data to train_not_strict_98.fasta
Saving test data to test_not_strict_98.fa

In [70]:
!cat train.fasta | grep ">" | wc

 457002  457002 3670800


In [71]:
!cat test.fasta | grep ">" | wc

 114252  114252  917812


We can see that now the number of the sequences are 364.

>sp|A0A1Z0YU59|MAMB1_DENAN Mambaquaretin-1 OS=Dendroaspis angusticeps OX=8618 PE=1 SV=2
RPSFCNLPVKPGPCNGFFSAFYYSQKTNKCHSFTYGGCKGNANRFSTIEKCRRTCVG

>sp|B2G331|VKT2B_HETCR TauPI-stichotoxin-Hcr2b OS=Heteractis crispa OX=175771 PE=1 SV=1
MKGTFLICLILIAGFSFKSTQAGSICLEPKVVGPCTAYFRRFYFDSETGKCTVFIYGGCE
GNGNNFETLRACRAICRA

>sp|B7S4N9|VKT_OXYSC Kunitz-type serine protease inhibitor taicotoxin OS=Oxyuranus scutellatus scutellatus OX=8667 PE=1 SV=1
MSSGGLLLLLGLLTLWEVLTPVSSKDRPKFCHLPPKPGPCRAAIPRFYYNPHSKQCEKFI
YGGCHGNANSFKTPDECNYTCLGVSLPK

>sp|I2G9B4|VKT_MACLN Kunitz-type serine protease inhibitor PIVL OS=Macrovipera lebetina transmediterranea OX=384075 PE=1 SV=1
MSSGGLLLLLGLLTLWAELTPVSSQDRPKFCYLPADPAECNAYMPRFYYDSASNKCKEFI
YGGCRGNANNFKNRAECRHTCVASRKGIQPRIASN

>sp|M9PE65|AXO_DROME Axotactin OS=Drosophila melanogaster OX=7227 GN=axo PE=1 SV=1
MAFPYIWALLPLICSASGLSLPNMTSTDAVVAGGGILPILVAGNPGNLGSSNMSLSGGGG
LAGSSTGGQSLPDTGGGNSAGGSPAGGSSGTGGGGSNSGISGNNSAMIQGQKSNQYEKCA
GPGDPGPCKQYIYKWRYEPTTNECTNFIWGGCEGNPQNR